In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install dgl==0.6.1
!pip install torch==1.9.1

     |████████████████████████████████| 831.4 MB 5.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.9.1 which is incompatible.


In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import pandas as pd
import networkx as nx
import torch as th

Using backend: pytorch


In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/PP-Pathways_ppi.csv', header = None)
#ds = dgl.data.CSVDataset('/path/to/dataset')

In [4]:
G=nx.from_pandas_edgelist(df,  0,  1,create_using=nx.Graph())
g=dgl.from_networkx(G)
g=dgl.to_bidirected(g)

In [5]:
print("type: ",g.is_multigraph)
print("No of nodes: ",g.number_of_nodes())
print("No of edges: ",g.number_of_edges())

type:  False
No of nodes:  21557
No of edges:  680989


In [6]:
# creating features for dataset i,e; `Degree`
feature=g.in_degrees()
feature=torch.reshape(feature, (21557,1))
g.ndata['feat']=feature

#### formulate the link prediction problem as a binary classification problem as follows:

- Treat the edges in the graph as positive examples.
- Sample a number of non-existent edges (i.e. node pairs with no edges between them) as negative examples.
- Divide the positive examples and negative examples into a training set and a test set.
- Evaluate the model with any binary classification metric such as Area Under Curve (AUC).

- randomly pick 10% of the edges for positive examples in the test set, and leave the rest for the training set. Then samples the same number of edges for negative examples in both sets.

In [7]:
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)  #  Disorder the order 

test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))  #  Use a full 1 vector , And corresponding u and v To construct adjacency matrix 
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())  #  Obtain the negative sampling adjacency matrix  adj.todense() Indicates that the sparse matrix adj Become a dense matrix 
neg_u, neg_v = np.where(adj_neg != 0)  #  Negative sampling on adjacency matrix 

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [8]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [9]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [45]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [11]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [12]:

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)



In [39]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 24)
pred = DotPredictor()

In [40]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(350):
    # forward
    h = model(train_g, train_g.ndata['feat'].float())
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))


In epoch 0, loss: 442800.09375
In epoch 5, loss: 122178.1171875
In epoch 10, loss: 25685.23046875
In epoch 15, loss: 19105.470703125
In epoch 20, loss: 17417.9453125
In epoch 25, loss: 8113.88134765625
In epoch 30, loss: 3722.365966796875
In epoch 35, loss: 3048.437255859375
In epoch 40, loss: 2821.181884765625
In epoch 45, loss: 1980.25732421875
In epoch 50, loss: 1224.3150634765625
In epoch 55, loss: 918.4205932617188
In epoch 60, loss: 816.375244140625
In epoch 65, loss: 754.0439453125
In epoch 70, loss: 686.319091796875
In epoch 75, loss: 620.6762084960938
In epoch 80, loss: 569.4276733398438
In epoch 85, loss: 533.9817504882812
In epoch 90, loss: 506.2844543457031
In epoch 95, loss: 482.24755859375
In epoch 100, loss: 462.0334777832031
In epoch 105, loss: 442.71514892578125
In epoch 110, loss: 425.3411865234375
In epoch 115, loss: 408.9414367675781
In epoch 120, loss: 393.9778137207031
In epoch 125, loss: 379.5087890625
In epoch 130, loss: 365.3929443359375
In epoch 135, loss: 352

In [41]:
# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

AUC 0.8093542052583326
